In this project, you will be working with data extracted from famous recommender systems type datasets: you are provided with a large set of interactions between users (persons)  and items (movies). Whenever a user "interacts" with an item, it watches the movie and gives a mark or "rating": you can interpret a rating of "1" as a "like", a rating of "-1" as a "dislike" and a rating of "0" as a neutral "meh" rating.

In this exercise, we will **not** be performing the recommendation task per se. Instead, we will identify *anomalous users*. In the dataset that you are provided with, some of the data was corrupted. Whilst most of the data comes from real life user-item interactions from a famous movie rating website, some "users" are anomalous: they were generated by me according to some undisclosed procedure.

You are provided with two data frames: the first one ("ratings") contains the interactions provided to you, and the second one ("labels") contains the labels for the users.

As you can see, the three columns in "ratings" correspond to the user ID, the item ID and the rating. Thus, each row of "ratings" contains a single interaction. For instance, if the row "142, 152, 1" is present, this means that the user with ID 142 has given the movie 152 a positive rating of "1" ("like").

The dataframe "labels" has two columns. In the first column we have the user ids, whilst the second column contains the labels. A label of 1 indicates that the user is fake (generated by me), whilst a label of 0 denotes a natural user (coming from real life interactions).

For instance, if the labels matrix contains the line "142, 1", it means that all of the ratings given by the user with id 142 are fake. This means all lines in the dataframe "ratings" which start with the userID 142 correspond to fake interactions.

Your task is to be able to classify unseen instances as either anomalies or non anomalies (guess whether they are real users or if they were generated by me).

There are **far more** normal users than anomalies in the dataset, which makes this a very heavily **unbalanced dataset**. Thus, accuracy will not be a good measure of performance, since simply predicting that every user is normal will give good accuracy. Thus, we need to use some other evaluation metrics.

THE **EVALUATION METRICS** are:  THE **AUC** (AREA UNDER CURVE), the **PRECISION**, THE **RECALL**, and the **F1 score**. The **main metric** will be the **AREA UNDER CURVE**, and it will by default be used to rank teams. This means your programs should return an **anomaly score** for each user (the higher the score, the more likely the model think the sample is anomalous).

# Training Methodology
## Data Preprocessing:
1. Split combined data set into train (train_df) and test (test_df) (80/20)
2. Aggregate training data, add more features into it
3. Split training data into train (df_X_train/df_y_train) and validation (df_X_val/df_y_val) sets (80/20)
4. Normalize training set and validation set

## Model Training:
1. Apply PCA to training set and validation set
2. Conduct SVM training on training set, then apply it onto the validation set, to get the AUC score
3. Find optimal threshold
4. Find out model performance using the evaluation metrics (Accuracy, Precision, Recall, F1 score)

## Model Testing:
1. Using test_df that we split from the beginning, apply PCA and conduct aggregation on it
2. Apply the SVM model onto test_df and get the AUC score

## Model Testing, using fourth data set, to generate csv file:
1. Using test_data (fourth_batch_likes.npz), apply PCA and conduct aggregation on it
2. Generate predictions and save it into csv file

---
# Imports

In [738]:
import pandas as pd

import numpy as np
from numpy import arange
from numpy import argmax

from scipy.stats import kurtosis
from scipy.stats import iqr
from scipy.stats import skew
from scipy.stats import entropy
from scipy.stats import mode
from scipy.stats import gmean
from scipy.stats import hmean
from scipy.stats import sem
from scipy.stats import gstd


from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

---
## Data Preprocessing:
1. Combine all data sets together
2. Split combined data set into train (train_df) and test (test_df) (80/20)
3. Aggregate training data, add more features into it
4. Split training data into train (df_X_train/df_y_train) and validation (df_X_val/df_y_val) sets (80/20)
5. Normalize training set and validation set

### 1. Combine all data sets together

In [739]:
# load the first batch
filenames = ["first_batch_with_labels_likes.npz", "second_batch_with_labels_likes.npz", "third_batch_with_labels_likes.npz", "fourth_batch_with_labels_likes.npz"]
def merge_datasets(filenames: list[str]) -> pd.DataFrame:
    merged_df = pd.DataFrame()
    for s in filenames:
        data = np.load(s)
        X1, y1 = data["X"], data["y"]
        XX1, yy1 = pd.DataFrame(X1), pd.DataFrame(y1)
        XX1.rename(columns={0: "user", 1: "item", 2: "rating"}, inplace=True)
        yy1.rename(columns={0: "user", 1: "label"}, inplace=True)
        merged_df = pd.concat([merged_df, pd.merge(XX1, yy1, on='user')], ignore_index=True)
    return merged_df

combined_data = merge_datasets(filenames)
combined_data

,user,item,rating,label
0,1220,6,0,0
1,1220,21,1,0
2,1220,31,0,0
3,1220,33,0,0
4,1220,35,-1,0
...,...,...,...,...
786631,5071,141,-1,1
786632,5071,158,-1,1
786633,5071,176,0,1
786634,5071,181,-1,1


### 2. Split combined data set into train (train_df) and test (test_df) (80/20)

In [740]:
# Get a list of unique users
unique_users = combined_data['user'].unique()

# Split the data into train and test sets (80/20)
train, test = train_test_split(unique_users, test_size=0.2, random_state=42)

# Create dataframes for train and test sets
train_df = combined_data[combined_data['user'].isin(train)]
test_df = combined_data[combined_data['user'].isin(test)]

# Count the number of users in the train and test sets
print(f'train size: {len(train_df)}\ntest size: {len(test_df)}')


train size: 625256
test size: 161380


## 3. Aggregate training data, add more features into it
### I will aggregate the training data first, then split it into train and validation later

In [741]:
def agg_rating_stats(X, y):
    unique_items = X.item.unique() #unique movies
    columns = np.append(unique_items, 'label')
    result = pd.DataFrame(columns=columns)
    lst = []
    total = len(X.item.unique())

    for user in X.user.unique():
        ratings = X[X['user'] == user]['rating'] # ratings by single user
        ratings_df = X[X['user'] == user] # user -> item -> rating df for single user
        ratings_df.loc[:, 'item'] = ratings_df['item'].astype(str) # convert movie id from number to string
        ratings_dict = dict(zip(ratings_df.item, ratings_df.rating))
        label = y[y['user'] == user].iloc[0]['label']
        ratings_dict['label'] = label
        ratings_dict['user'] = user
        ratings_dict['nan_count'] = total - len(ratings)
        ratings_dict['nan_proportion_of_total'] = (total - len(ratings)) / total
        ratings_dict['rating_count'] = len(ratings)
        ratings_dict['rating_proportion_of_total'] = len(ratings) / total
        ratings_dict['rating_mean'] = np.mean(ratings)
        ratings_dict['rating_sem'] = sem(ratings)
        ratings_dict['rating_sd'] = np.std(ratings)
        ratings_dict['rating_variance'] = np.var(ratings)
        ratings_dict['rating_sum'] = sum(ratings)
        ratings_dict['negative_count'] = sum(rating < 0 for rating in ratings)
        ratings_dict['negative_proportion'] = sum(rating < 0 for rating in ratings) / len(ratings)
        ratings_dict['negative_proportion_of_total'] = sum(rating < 0 for rating in ratings) / total
        ratings_dict['neutral_count'] = sum(rating == 0 for rating in ratings)
        ratings_dict['neutral_proportion'] = sum(rating == 0 for rating in ratings) / len(ratings)
        ratings_dict['neutral_proportion_of_total'] = sum(rating == 0 for rating in ratings) / total
        ratings_dict['positive_count'] = sum(rating > 0 for rating in ratings)
        ratings_dict['positive_proportion'] = sum(rating > 0 for rating in ratings) / len(ratings)
        ratings_dict['positive_proportion_of_total'] = sum(rating > 0 for rating in ratings) / total
        ratings_dict['skew'] = skew(ratings)
        ratings_dict['entropy'] = entropy([ratings_dict['negative_proportion_of_total'], ratings_dict['positive_proportion_of_total'], 1-ratings_dict['negative_proportion_of_total']- ratings_dict['positive_proportion_of_total']])
        ratings_dict['kurtosis'] = kurtosis(ratings)
        ratings_dict['iqr'] = iqr(ratings)
        lst.append(ratings_dict)

    result = pd.concat([result, pd.DataFrame(lst)], ignore_index=True)
    result = result.fillna(-1)
    return result

In [742]:
# Split training data into X and y
XX_train = train_df[['user', 'item', 'rating']]
yy_train = train_df[['user', 'label']]

# Apply aggregation function to training data
df1 = agg_rating_stats(XX_train, yy_train)
# df1


C:\Users\jinha\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\jinha\AppData\Local\Temp/ipykernel_15672/1597720225.py:34: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  ratings_dict['skew'] = skew(ratings)
C:\Users\jinha\AppData\Local\Temp/ipykernel_15672/1597720225.py:36: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  ratings_dict['kurtosis'] = kurtosis(ratings)


In [743]:
### helper method to convert probabilities & thresholds to binary values
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')

### 4. Split training data into train (df_X_train/df_y_train) and validation (df_X_val/df_y_val) sets (60/20)

In [744]:
df1_XXX = df1.drop(['user', 'label'], axis=1)
df1_yyy = df1.label

# Columns
X_cols = df1_XXX.columns
y_col = df1_XXX.columns

# Split training set into train and validation sets (80/20)
df_X_train, df_X_val, df_y_train, df_y_val = train_test_split(df1_XXX, df1_yyy, stratify=df1_yyy, test_size=0.25, random_state=42) #Change to 25

### 5. Normalize training set and validation set

In [745]:
# Normalize input variables
scaler = preprocessing.StandardScaler()

df_X_train_normalized = pd.DataFrame(scaler.fit_transform(df_X_train), columns=X_cols)
df_X_val_normalized = pd.DataFrame(scaler.transform(df_X_val), columns=X_cols) # good

---
## Model Training:
1. Apply PCA to training set and validation set
2. Conduct SVM training on training set, then apply it onto the validation set, to get the AUC score
3. Find optimal threshold
4. Find out model performance using the evaluation metrics (Accuracy, Precision, Recall, F1 score)

### 1. Apply PCA to training set and validation set

### 2. Conduct SVM training on training set, then apply it onto the validation set, to get the AUC score

In [781]:
# pc=PCA(n_components=19, svd_solver='full')
kpc = KernelPCA(n_components=34, eigen_solver='dense', kernel='rbf',gamma=0.00041)
X_train_PC = kpc.fit_transform(df_X_train_normalized)
X_val_PC = kpc.transform(df_X_val_normalized)

In [783]:
# Fit model 
svc = svm.SVC(class_weight='balanced', probability=True, C=1.5, gamma=21.85, shrinking=False, decision_function_shape='ovr', break_ties=True)
svc.fit(X_train_PC, df_y_train)
predictions = svc.predict_proba(X_val_PC)[:, 1]
auc = roc_auc_score(df_y_val, predictions)
print(auc)
#0.959352

0.9594395493283259


### 3. Find optimal threshold

In [748]:
### find optimal threshold
thresholds = arange(0, 1, 0.001)
scores = [f1_score(df_y_val, to_labels(predictions, t)) for t in thresholds]
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))
svc_threshold = thresholds[ix]

Threshold=0.403, F-Score=0.80851


### 4. Find out model performance using the evaluation metrics (Accuracy, Precision, Recall, F1 score)

In [749]:
### evaluate model performance
predicted_labels = to_labels(predictions, svc_threshold)

conf_matrix = confusion_matrix(predicted_labels, df_y_val)

# Retrieve TN, FN, TP and FP
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

# accuracy = (correct predictions) / (all predictions)
accuracy = (tn + tp) / (tn + fn + tp + fp)
print('accuracy =', accuracy)

# precision = proportion of correct 'true' predictions
precision = tp / (tp + fp)
print('precision =', precision)

# recall = proportion of true records correctly identified
recall = tp / (tp + fn)
print('recall =', recall)

# f1_score = 2*P*R / P+R
f1 = (2 * precision * recall) / (precision + recall)
print('f1 score =', f1)

accuracy = 0.9117647058823529
precision = 0.8837209302325582
recall = 0.7450980392156863
f1 score = 0.8085106382978724


---
## Model Testing:
1. Using test_df that we split from the beginning, apply PCA and conduct aggregation on it
2. Apply the SVM model onto test_df and get the AUC score

### 1. Using test_df that we split from the beginning, apply PCA and conduct aggregation on it

In [750]:
# Helper method to process test data (No label)
def agg_test_stats(X):
    result = pd.DataFrame(columns=X_cols)
    lst = []
    total = len(X.item.unique())

    for user in X.user.unique():
        ratings = X[X['user'] == user]['rating']
        ratings_df = X[X['user'] == user]
        ratings_df.loc[:, 'item'] = ratings_df['item'].astype(str)
        ratings_dict = dict(zip(ratings_df.item, ratings_df.rating))
        ratings_dict['user'] = user
        ratings_dict['nan_count'] = total - len(ratings)
        ratings_dict['nan_proportion_of_total'] = (total - len(ratings)) / total
        ratings_dict['rating_count'] = len(ratings)
        ratings_dict['rating_proportion_of_total'] = len(ratings) / total
        ratings_dict['rating_mean'] = np.mean(ratings)
        ratings_dict['rating_sem'] = sem(ratings)
        ratings_dict['rating_sd'] = np.std(ratings)
        ratings_dict['rating_variance'] = np.var(ratings)
        ratings_dict['rating_sum'] = sum(ratings)
        ratings_dict['negative_count'] = sum(rating < 0 for rating in ratings)
        ratings_dict['negative_proportion'] = sum(rating < 0 for rating in ratings) / len(ratings)
        ratings_dict['negative_proportion_of_total'] = sum(rating < 0 for rating in ratings) / total
        ratings_dict['neutral_count'] = sum(rating == 0 for rating in ratings)
        ratings_dict['neutral_proportion'] = sum(rating == 0 for rating in ratings) / len(ratings)
        ratings_dict['neutral_proportion_of_total'] = sum(rating == 0 for rating in ratings) / total
        ratings_dict['positive_count'] = sum(rating > 0 for rating in ratings)
        ratings_dict['positive_proportion'] = sum(rating > 0 for rating in ratings) / len(ratings)
        ratings_dict['positive_proportion_of_total'] = sum(rating > 0 for rating in ratings) / total
        ratings_dict['skew'] = skew(ratings)
        ratings_dict['entropy'] = entropy([ratings_dict['negative_proportion_of_total'], ratings_dict['positive_proportion_of_total'], 1-ratings_dict['negative_proportion_of_total']- ratings_dict['positive_proportion_of_total']])
        ratings_dict['kurtosis'] = kurtosis(ratings)
        ratings_dict['iqr'] = iqr(ratings)
        lst.append(ratings_dict)

    result = pd.concat([result, pd.DataFrame(lst)], ignore_index=True)
    result = result.fillna(-1)  # arbitrarily fill empty values
    return result

In [751]:
XX_test = test_df[['user', 'item', 'rating']]
yy_test = test_df[['user', 'label']]

# XX_test.count()
# yy_test.count()

# Apply aggregation function to test data
df2 = agg_test_stats(XX_test)
# df2

C:\Users\jinha\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\jinha\AppData\Local\Temp/ipykernel_15672/2639459121.py:31: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  ratings_dict['skew'] = skew(ratings)
C:\Users\jinha\AppData\Local\Temp/ipykernel_15672/2639459121.py:33: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  ratings_dict['kurtosis'] = kurtosis(ratings)


In [752]:
XX_test_1 = df2[df2.columns.intersection(X_cols)]
XX_test_normalized = scaler.transform(XX_test_1)
# XX_test_poly = poly.transform(XX_test_scaled)
XX_test_PC = kpc.transform(XX_test_normalized) #pc from training. Good

C:\Users\jinha\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KernelPCA was fitted with feature names
  warnings.warn(


In [753]:
# Remove duplicate rows in yy_test
yy_test = yy_test.drop_duplicates(subset=['user'])

### 2. Apply the SVM model onto test_df and get the AUC score

In [754]:
predictions = svc.predict_proba(XX_test_PC)[:, 1]

# Evaluation metrics: AUC, Precision, Recall, F1
roc_auc_score(yy_test['label'], predictions)

0.956306819402953

---
## Model Testing, using fourth data set, to generate csv file:
1. Using test_data (fourth_batch_likes.npz), apply PCA and conduct aggregation on it
2. Generate predictions and save it into csv file

In [755]:
test_data = np.load("fifth_batch_likes.npz")
df_X_test = pd.DataFrame(test_data["X"])
df_X_test.rename(columns={0: "user", 1: "item", 2: "rating"}, inplace=True)
df_X_test

,user,item,rating
0,5233,0,1
1,5233,9,0
2,5233,19,1
3,5233,24,0
4,5233,25,0
...,...,...,...
285563,6973,456,0
285564,6973,461,-1
285565,6973,477,1
285566,6973,486,-1


### 1. Using test_data (fourth_batch_likes.npz), apply PCA and conduct aggregation on it

In [756]:
X_test = agg_test_stats(df_X_test)
X_test = X_test.sort_values(by=['user'], ascending=True)
X_test

C:\Users\jinha\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\jinha\AppData\Local\Temp/ipykernel_15672/2639459121.py:31: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  ratings_dict['skew'] = skew(ratings)
C:\Users\jinha\AppData\Local\Temp/ipykernel_15672/2639459121.py:33: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  ratings_dict['kurtosis'] = kurtosis(ratings)


,6,21,31,33,35,40,54,56,58,60,...,neutral_proportion,neutral_proportion_of_total,positive_count,positive_proportion,positive_proportion_of_total,skew,entropy,kurtosis,iqr,user
893,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,0.0,1.0,-1.0,...,0.388889,0.014042,16,0.444444,0.016048,-0.483419,0.118905,-1.009389,1.0,5100.0
1354,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,...,0.309859,0.022066,21,0.295775,0.021063,0.184616,0.229560,-1.506980,2.0,5101.0
1223,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,0.0,1.0,-1.0,...,0.292818,0.053159,123,0.679558,0.123370,-1.180968,0.404483,0.373424,1.0,5102.0
809,-1.0,1.0,1.0,0.0,-1.0,-1.0,0.0,1.0,1.0,1.0,...,0.278912,0.082247,184,0.625850,0.184554,-1.092831,0.600426,-0.022625,1.0,5103.0
564,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,...,0.098361,0.006018,53,0.868852,0.053159,-2.813062,0.222070,7.260394,0.0,5104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,1.0,1.0,-1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.402778,0.203611,243,0.482143,0.243731,-0.609432,0.760435,-0.724940,1.0,7095.0
389,-1.0,-1.0,1.0,-1.0,-1.0,0.0,-1.0,-1.0,1.0,-1.0,...,0.518797,0.069208,29,0.218045,0.029087,0.059781,0.282563,-0.914102,1.0,7096.0
1057,1.0,1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,...,0.333333,0.067202,111,0.552239,0.111334,-0.824700,0.456293,-0.533641,1.0,7097.0
1882,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.276316,0.021063,25,0.328947,0.025075,0.125341,0.251462,-1.598751,2.0,7098.0


In [757]:
X_test_1 = X_test[X_test.columns.intersection(X_cols)]
X_test_scaled = scaler.transform(X_test_1)
X_test_poly = poly.transform(X_test_scaled)
X_test_PC = pc.transform(X_test_poly)

NameError: name 'poly' is not defined

### 2. Generate predictions and save it into csv file

In [758]:
### helper method to generate output csv
def generate_test_output(users, predictions, opt_threshold, file_name):
    predictions =  predictions - opt_threshold 
    d = {'user': users, 'anomaly_score': predictions}
    output_df = pd.DataFrame(data=d)
    output_df = output_df.astype({'user':'int'})
    output_df.reset_index(drop=True)
    output_df = output_df.sort_values(by=['user'])
    output_df.to_csv(file_name, index=False, header=False)
    return output_df

In [759]:
users = X_test.user

In [760]:
predictions = svc.predict_proba(X_test_PC)[:, 1]
predictions.shape
output_csv = generate_test_output(users, predictions, svc_threshold, file_name='answer.csv')
output_csv

NameError: name 'X_test_PC' is not defined

---